In [ ]:
#%% Initial imports etc
import numpy
from numpy.linalg import norm
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import os
import sys
import shutil

from ccpi.utilities.jupyter import *
from ccpi.utilities.display import *

#%% Use the 'pet' prefix for all STIR-based SIRF functions
# This is done here to explicitly differentiate between SIRF pet functions and 
# anything else.
import sirf.STIR as pet
from sirf.Utilities import examples_data_path

pet.AcquisitionData.set_storage_scheme('memory')

#%% Go to directory with input files
# Adapt this path to your situation (or start everything in the relevant directory)
os.chdir(examples_data_path('PET'))

#%% Copy files to a working folder and change directory to where these files are.
# We do this to avoid cluttering your SIRF files. This way, you can delete 
# working_folder and start from scratch.
if False:
    shutil.rmtree('working_folder/brain',True)
    shutil.copytree('brain','working_folder/brain')
os.chdir('working_folder/brain')

#%% Read in images
# Here we will read some images provided with the demo using the ImageData class.
# These are in Interfile format. (A text header pointing to a .v file with the binary data).
image = pet.ImageData('emission.hv')
mu_map = pet.ImageData('attenuation.hv')

In [ ]:
direction = 0
islicer(image,direction, cmap='viridis')

In [ ]:
#%% Create a SIRF acquisition model
# We will use the ray-tracing matrix here as our simple PET model.
# There is more to the accquisition model, but that's for another demo.
am = pet.AcquisitionModelUsingRayTracingMatrix()
# Ask STIR to use 5 LORs per sinogram-element
am.set_num_tangential_LORs(5)

#%% Specify sinogram dimensions
# We need to say what scanner to use, what dimensions etc.
# You do this by using existing PET data as a 'template'. 
# Here, we read a file supplied with the demo as an AcquisitionData object
templ = pet.AcquisitionData('template_sinogram.hs')
# Now set-up our acquisition model with all information that it needs about the data and image.
am.set_up(templ,image)

In [ ]:
# ADD NOISE

sino = am.direct(image)

def add_noise(counts, sinogram):
    sino_arr = sinogram.as_array()
    minmax = (sino_arr.min(), sino_arr.max())
    if counts > 0 and counts <= 1:
        counts = counts * (minmax[1] - minmax[0])
    elif isinstance (counts, int):
        pass
       
    sino_arr = counts * ((sino_arr -minmax[0]) / (minmax[1]-minmax[0]))
    noisy_counts = sinogram * 0.
    noisy_counts.fill( numpy.random.poisson(sino_arr) )
    
    return noisy_counts


minmax = sino.as_array().min(), sino.as_array().max()

noisy_counts = add_noise(1, sino)

s0 = islicer(noisy_counts.as_array()[0], 0, cmap='inferno_r')
s1 = islicer(sino.as_array()[0], 0, cmap='inferno_r')
link_islicer(s0,s1)

#del sino

In [ ]:
import sirf.Reg as Reg
from scipy.spatial.transform import Rotation as R

def get_resampler(directions, angles, degrees=True ):
    '''example input 'zy', [87,13], degrees=True'''
    r = R.from_euler(directions, angles, degrees=degrees)

    mat = r.as_dcm()



    tm = Reg.AffineTransformation()
    mat4 = tm.as_array()

    for i in range(3):
        for j in range(3):
            mat4[i][j] = mat[i][j]

    tm = Reg.AffineTransformation(mat4)

    mat = tm.as_array()

    resampler = Reg.NiftyResample()
    resampler.set_reference_image(image)
    resampler.set_floating_image(image)
    resampler.add_transformation(tm)
    resampler.set_padding_value(0)
    resampler.set_interpolation_type_to_linear()
    
    return resampler

In [ ]:
# create different motion state
rotations = [[-1.2,3.],[1.2,-3.],[0.,-5.], [.2,2.]]
rotations = [ [10 * rot[0],rot[1]] for rot in rotations ]

resamplers = [ get_resampler('zy', rot, degrees=True) for rot in rotations ]

# create the new AcquisitionData for the motion states
rotated_sinos = []

for rot, resampler in zip(*(rotations, resamplers)):
    # new ImageData
    out = resampler.direct(image)
    # new AcquisitionData
    rs = am.direct(out)
    # add noise
    rs = add_noise(1,rs)
    rotated_sinos.append(rs)

del out, rs


In [ ]:
#s0 = islicer(acquired_data.as_array()[0], 0, cmap='viridis')

a = rotated_sinos[0]-rotated_sinos[1]
print (type(a))

s1 = islicer(resamplers[0].direct(image).as_array(), 0, cmap='viridis_r')
s2 = islicer(resamplers[1].direct(image).as_array(), 0, cmap='viridis_r')
s3 = islicer((rotated_sinos[0]-rotated_sinos[1]).as_array()[0],0,cmap='viridis')
link_islicer(s1,s2)

In [ ]:
from ccpi.optimisation.operators import CompositionOperator, BlockOperator, LinearOperator


C = [ CompositionOperator(am, resampler, preallocate=True) for resampler in resamplers ]
# C = [ am for _ in resamplers ]
# norms = [ LinearOperator.PowerMethod(op, 25)[0] for op in C ]


In [ ]:
# n = [nn[0] for nn in norms]
# norms = n
# print (norms, sum(norms))

from ccpi.plugins.regularisers import FGP_TV
#FGP_TV??

In [ ]:
from ccpi.optimisation.algorithms import PDHG
from ccpi.optimisation.functions import KullbackLeibler, IndicatorBox, BlockFunction
from ccpi.optimisation.operators import BlockOperator
from ccpi.plugins.regularisers import FGP_TV

#regularisation parameters for TV
# 
r_alpha = 5e-1
r_iterations = 500
r_tolerance = 1e-7
r_iso = 0
r_nonneg = 1
r_printing = 0

TV = FGP_TV(r_alpha, r_iterations, r_tolerance, r_iso,r_nonneg,r_printing,'cpu')

motion = True
if motion:
    #noisy_counts is the GT forward projected + noise
    kl = [ KullbackLeibler(b=rotated_sino, eta=(rotated_sino * 0 + 1e-5)) for rotated_sino in rotated_sinos ] 
    f = BlockFunction(*kl)
    K = BlockOperator(*C)
    normK = K.norm(iterations=2)
    #normK = numpy.sqrt(sum( norms ))
else:
    f = KullbackLeibler(b=noisy_counts, eta=(noisy_counts * 0 + 1e-5))
    K = am
    normK = LinearOperator.PowerMethod(am, 25)[0]

In [ ]:
rotated_sino.shape

In [ ]:
f(K.direct(image))

In [ ]:
sigma = 1/normK
tau = 1/normK  

G = IndicatorBox(lower=0)
# G = TV
# print (f(acquired_data*0.+1e-5))
# print (f(acquired_data*0.))

In [ ]:
def do_nothing(self):
    return 0.
setattr(PDHG, 'update_objective', do_nothing)

In [ ]:
# Setup and run PDHG


pdhg = PDHG(f = f, g = G, operator = K, sigma = sigma, tau = tau, 
            max_iteration = 1000,
            update_objective_interval = 4)

In [ ]:
na = numpy.zeros((2,2))
a = [na, na]
na += 1

print (a)

In [ ]:
pdhg.run(8, verbose=False)

pdhg_recon = pdhg.get_output()     

In [ ]:
pdhg.max_iteration = 2000
#pdhg.run()

pdhg_l1_recon = pdhg.get_output()     


In [ ]:
pdhg_l1_recon = pdhg.get_output()
iM, im = image.as_array().max(), image.as_array().min()
rM, rm = pdhg_l1_recon.as_array().max(), pdhg_l1_recon.as_array().min()

i_scaled = ((image -im) / (iM-im))
r_scaled = ((pdhg_l1_recon -rm) / (rM-rm))
s0 = islicer(i_scaled, 0, cmap='inferno')
s1 = islicer(r_scaled, 0, cmap='inferno')

link_islicer(s0, s1)

In [ ]:
#pdhg.get_output().write('PDHG_MCIR_noNoise_Motion_1000it_TV0.h')